<a href="https://colab.research.google.com/github/PadawanXXVI/PadawanXXVI/blob/main/spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎧 Spotify Data Storytelling — Semana 2
### Curso: Tecnologia em Ciência de Dados – Senac DF  
### Disciplina: Deep Learning e Laboratório de Inovação IV  
**Professor:** Alexsander Holanda Barreto  
**Autor:** Anderson de Matos Guimarães  
**Notebook:** spotify.ipynb  
**Objetivo da Semana 2:**  
Realizar a coleta, limpeza e transformação de dados musicais via API do Spotify, explorando playlists nacionais e regionais para identificar padrões de popularidade e comportamento musical no Brasil.


In [1]:
# Instalação (somente se spotipy não estiver presente)
!pip install spotipy --quiet

# Importação das bibliotecas principais
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 7.5 MB/s eta 0:00:00


In [2]:
# Instalação
!pip install spotipy --quiet

# Importação das bibliotecas principais
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [5]:
# ⚙️ Autenticação via Spotipy usando Secrets do Colab
from google.colab import userdata
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Carrega os secrets armazenados na aba "Chaves secretas"
client_id = userdata.get("SPOTIPY_CLIENT_ID")
client_secret = userdata.get("SPOTIPY_CLIENT_SECRET")

if not client_id or not client_secret:
    raise ValueError("⚠️ As variáveis SPOTIPY_CLIENT_ID e SPOTIPY_CLIENT_SECRET não foram encontradas nos secrets do Colab.")

# Configura autenticação segura
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

print("✅ Autenticação concluída com sucesso!")


✅ Autenticação concluída com sucesso!


In [16]:
# 🔎 Listagem das categorias disponíveis no Spotify Brasil

categories_data = sp.categories(country="BR", limit=50)
categories = {c["name"]: c["id"] for c in categories_data["categories"]["items"]}

print(f"✅ Total de categorias disponíveis: {len(categories)}\n")
print("Categorias disponíveis no Brasil:")
for name in categories.keys():
    print(f"- {name}")

# Exibe as 10 primeiras como referência
print("\n🧭 Primeiras categorias retornadas:")
print(list(categories.keys())[:10])


✅ Total de categorias disponíveis: 50

Categorias disponíveis no Brasil:
- Made For You
- New Releases
- Hip-Hop
- Country
- Pop
- Latin
- Charts
- Rock
- R&B
- Dance/Electronic
- Indie
- Fall
- Halloween
- Diwali
- Workout
- Mood
- Party
- Student
- Love
- Disney
- Netflix
- Chill
- Sleep
- Discover
- Jazz
- Metal
- Christian & Gospel
- Música Mexicana
- Happy Holidays
- Classical
- Decades
- Kids & Family
- In the car
- At Home
- EQUAL
- Spotify CLASSICS
- GLOW
- Frequency
- TV & Movies
- Trending
- Folk & Acoustic
- K-pop
- Punk
- Blues
- Soul
- Alternative
- Anime
- Instrumental
- RADAR
- Fresh Finds

🧭 Primeiras categorias retornadas:
['Made For You', 'New Releases', 'Hip-Hop', 'Country', 'Pop', 'Latin', 'Charts', 'Rock', 'R&B', 'Dance/Electronic']


🎯 Total de gêneros para busca: 20

🎶 Buscando faixas do gênero: sertanejo
🎶 Buscando faixas do gênero: sertanejo universitário
🎶 Buscando faixas do gênero: forró
🎶 Buscando faixas do gênero: modão
⚠️ Nenhum resultado para o gênero: modão
🎶 Buscando faixas do gênero: mpb
🎶 Buscando faixas do gênero: bossa nova
🎶 Buscando faixas do gênero: samba
🎶 Buscando faixas do gênero: pagode
🎶 Buscando faixas do gênero: funk
🎶 Buscando faixas do gênero: rap
🎶 Buscando faixas do gênero: trap
🎶 Buscando faixas do gênero: pop
🎶 Buscando faixas do gênero: rock
🎶 Buscando faixas do gênero: indie
🎶 Buscando faixas do gênero: gospel
🎶 Buscando faixas do gênero: eletrônica
🎶 Buscando faixas do gênero: country
🎶 Buscando faixas do gênero: reggae
🎶 Buscando faixas do gênero: jazz
🎶 Buscando faixas do gênero: blues

✅ Resumo de faixas coletadas:


,genero,qtd_faixas
0,sertanejo,50
1,sertanejo universitário,50
2,forró,50
3,mpb,50
4,bossa nova,50
5,samba,50
6,pagode,50
7,funk,50
8,rap,50
9,trap,50


💾 Dataset exportado: 950 faixas em /content/data/spotify_raw.csv


In [22]:
# 🔍 Estrutura do dataset (info)
import pandas as pd

df = pd.read_csv("/content/data/spotify_raw.csv")

print(f"✅ Dataset bruto carregado: {df.shape[0]} linhas e {df.shape[1]} colunas.\n")
df.info()


✅ Dataset bruto carregado: 950 linhas e 7 colunas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   genero           950 non-null    object
 1   faixa            950 non-null    object
 2   artista          950 non-null    object
 3   album            950 non-null    object
 4   data_lancamento  950 non-null    object
 5   popularidade     950 non-null    int64 
 6   duracao_ms       950 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 52.1+ KB


In [23]:
# 📈 Estatísticas iniciais do dataset
display(df.describe())


,popularidade,duracao_ms
count,950.000000,950.000000
mean,26.977895,214546.023158
std,36.140933,74588.239080
min,0.000000,5000.000000
25%,0.000000,171291.000000
50%,0.000000,206480.000000
75%,68.000000,250240.000000
max,100.000000,733000.000000


In [24]:
# 👀 Visualização inicial das faixas
print("🔹 Primeiras 5 entradas:")
display(df.head())

print("\n🔹 Últimas 5 entradas:")
display(df.tail())


🔹 Primeiras 5 entradas:


,genero,faixa,artista,album,data_lancamento,popularidade,duracao_ms
0,sertanejo,Arrasta Pé,Grupo Tradição,Entrevero de Vaneira,2018-04-11,0,263373
1,sertanejo,Americana,Mario Zan,Só Para Você / Americana,1953-01-01,0,185670
2,sertanejo,Meu Querido Lindo,Moacyr Franco,Contrastes,2020-11-30,0,144720
3,sertanejo,Tia Geralda,Vieira & Vieirinha,"Velha Guarda Sertaneja, Vol. 4",2014-10-21,0,208440
4,sertanejo,Eu Sou Feliz,Tchê Barbaridade,Tchê Amo - As Mais Românticas do Tchê Barbaridade,2022-02-15,0,209085



🔹 Últimas 5 entradas:


,genero,faixa,artista,album,data_lancamento,popularidade,duracao_ms
945,blues,Feeling Good,Nina Simone,I Put A Spell On You,1965-06-01,75,174000
946,blues,I Say a Little Prayer,Aretha Franklin,Aretha Now,1968-06-14,76,216773
947,blues,A Lot More Free,Max McNown,Wandering (Love Me Back),2024-09-06,77,198920
948,blues,Tears Dry On Their Own,Amy Winehouse,Back To Black (Deluxe Edition),2006,72,186440
949,blues,Hit the Road Jack,Ray Charles,True Genius,2021-09-10,74,119880


In [25]:
# 🧹 Limpeza inicial do dataset
df = df.drop_duplicates(subset=["faixa", "artista"])
df = df.dropna(subset=["faixa", "artista", "popularidade", "duracao_ms", "data_lancamento"])

print(f"✅ Após limpeza: {df.shape[0]} linhas restantes.")


✅ Após limpeza: 841 linhas restantes.


In [26]:
# ⚙️ Criação de colunas derivadas e normalização
df["data_lancamento"] = pd.to_datetime(df["data_lancamento"], errors="coerce")
df["ano_lancamento"] = df["data_lancamento"].dt.year
df["duracao_min"] = df["duracao_ms"] / 60000
df["popularidade_norm"] = (df["popularidade"] - df["popularidade"].min()) / (
    df["popularidade"].max() - df["popularidade"].min()
)


In [27]:
# ✍️ Padronização textual
for col in ["faixa", "artista", "album", "genero"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.title().str.strip()

display(df.sample(10))


,genero,faixa,artista,album,data_lancamento,popularidade,duracao_ms,ano_lancamento,duracao_min,popularidade_norm
706,Eletrônica,A.B.B.A,Tha_Guts,Plastic Noise,2018-04-24,0,183829,2018.0,3.063817,0.00
540,Pop,Cópia Proibida,Léo Foguete,Obrigado Deus,2024-10-26,80,159480,2024.0,2.658000,0.80
445,Rap,No Favors,Bezo Luciano,No Favors,2024-01-08,0,177214,2024.0,2.953567,0.00
712,Eletrônica,Jimmy'S Mines,Arthur Joly,Mugomango Elétrico Brasil 2002,2002-12-01,0,223608,2002.0,3.726800,0.00
72,Sertanejo Universitário,Festa Campeira - Ao Vivo,Garotos De Ouro,"Festa Gaúcha, Vol. 4 (Ao Vivo)",2018-02-08,0,197933,2018.0,3.298883,0.00
673,Gospel,Ser Crente É Assim,Cícero Nogueira,Adorador,2016-04-30,0,190014,2016.0,3.166900,0.00
679,Gospel,Lodate - Antifona,Antonio Parisi,Lodi E Vespri Della Seconda Domenica (Liturgia...,NaT,0,20760,NaN,0.346000,0.00
258,Samba,Tor Gharnu Mor Ghar,Santanu Sahu,Tor Gharnu Mor Ghar,2021-03-07,0,349924,2021.0,5.832067,0.00
851,Jazz,Fly Me To The Moon - 2008 Remastered,Frank Sinatra,Nothing But The Best (2008 Remastered),2008-05-13,66,147520,2008.0,2.458667,0.66
758,Country,Something In The Orange,Zach Bryan,Something In The Orange,2022-04-22,85,228013,2022.0,3.800217,0.85


In [28]:
# 💾 Exportação do dataset tratado
output_path = "/content/data/spotify_semana2_tratado.csv"
df.to_csv(output_path, index=False)
print(f"💾 Dataset tratado exportado com sucesso: {output_path}")


💾 Dataset tratado exportado com sucesso: /content/data/spotify_semana2_tratado.csv


In [29]:
# 📊 Verificação final do dataset tratado
df_clean = pd.read_csv("/content/data/spotify_semana2_tratado.csv")

print(f"✅ Dataset tratado: {df_clean.shape[0]} linhas.")
print("\n📈 Estatísticas pós-tratamento:")
display(df_clean.describe())

print("\n🧮 Nulos restantes:")
print(df_clean.isnull().sum())


✅ Dataset tratado: 841 linhas.

📈 Estatísticas pós-tratamento:


,popularidade,duracao_ms,ano_lancamento,duracao_min,popularidade_norm
count,841.000000,841.000000,790.000000,841.000000,841.000000
mean,27.242568,214969.803805,2012.808861,3.582830,0.272426
std,36.199489,76801.201435,14.448200,1.280020,0.361995
min,0.000000,5000.000000,1949.000000,0.083333,0.000000
25%,0.000000,169900.000000,2009.000000,2.831667,0.000000
50%,0.000000,206680.000000,2019.000000,3.444667,0.000000
75%,68.000000,249600.000000,2022.000000,4.160000,0.680000
max,100.000000,733000.000000,2025.000000,12.216667,1.000000



🧮 Nulos restantes:
genero                0
faixa                 0
artista               0
album                 0
data_lancamento      51
popularidade          0
duracao_ms            0
ano_lancamento       51
duracao_min           0
popularidade_norm     0
dtype: int64
